In [1]:
import numpy as np
import pandas as pd
import pathlib
from tqdm.notebook import tqdm

In [2]:
base_path = "/home/jakobs"

In [3]:
endpoints_md = pd.read_csv(f"{base_path}/BiHealth/onnx/endpoints.csv").drop(columns="Unnamed: 0").set_index("endpoint")#[["endpoint", "eligable", "n", "freq", "phecode", "phecode_string", "phecode_category", "sex"]]
endpoints_md

,eligable,n,freq,phecode,phecode_string,phecode_category,sex
endpoint,,,,,,,
OMOP_4306655,61256,3548,0.057921,4306655.00,All-Cause Death,Death,Both
phecode_002,60945,658,0.010797,2.00,Staphylococcus,ID,Both
phecode_002-1,61010,486,0.007966,2.10,Staphylococcus aureus,ID,Both
phecode_003,60757,1017,0.016739,3.00,Escherichia coli,ID,Both
phecode_004,60584,494,0.008154,4.00,Streptococcus,ID,Both
...,...,...,...,...,...,...,...
phecode_977-52,31669,520,0.016420,977.52,Hormone replacement therapy (postmenopausal),Rx,Female
phecode_977-7,60032,2231,0.037164,977.70,Long term (current) use of insulin or oral hyp...,Rx,Both
phecode_977-71,60936,472,0.007746,977.71,Long term (current) use of insulin,Rx,Both


In [8]:
import math
import warnings
from socket import gethostname

import numpy as np
import torch
import torchvision as tv
import torchmetrics

from retinalrisk.models.supervised import (
    ImageTraining
)
from retinalrisk.modules.head import MLPHead

def setup_training():
    
    def get_head(num_head_features, num_endpoints):

        cls = MLPHead

        return cls(
            num_head_features,
            num_endpoints,
            incidence=None,
            dropout=0.5,
            gradient_checkpointing=False,
            num_hidden =512, 
            num_layers =2,
            #loss=None,
        )
    
    base_path = "/home/jakobs"
    x = torch.load(f"{base_path}/BiHealth/ckpts/ckpt_partition_4")
    losses = x['hyper_parameters']["losses"]
    label_mapping = x['hyper_parameters']["label_mapping"]
    incidence_mapping = x['hyper_parameters']["incidence_mapping"]

    #encoder = tv.models.__dict__[args.model.encoder](pretrained=args.model.pretrained)
    weights = 'DEFAULT' 
    encoder = tv.models.__dict__["convnext_small"](weights=weights) 
    #print(encoder)

    outshape = 768 

    setattr(encoder.classifier, '2', torch.nn.Identity())

    head = get_head(num_head_features=768, num_endpoints = 1171)

    model = ImageTraining(encoder=encoder, head=head, losses=None, label_mapping=label_mapping, incidence_mapping=None, task="tte")

    return model

In [5]:
import torch.nn as nn
import PIL
from typing import Union
from random import choice
import torchvision as tv
import torchvision.transforms.functional as TF
from torchvision.transforms import InterpolationMode

import torch
import torchvision.transforms as transforms
from skimage.io import imread

crop_ratio = [0.66]
img_size_to_gpu = 420 # 420

class AdaptiveRandomCropTransform(nn.Module):
    def __init__(self,
                 crop_ratio: Union[list,float],
                 out_size: int,
                 interpolation=InterpolationMode.BILINEAR):
        super().__init__()
        self.crop_ratio = crop_ratio
        self.out_size = out_size
        self.interpolation = interpolation

    def forward(self, sample):
        input_size = min(sample.size)
        if isinstance(self.crop_ratio, list):
            crop_ratio = choice(self.crop_ratio)
        else:
            crop_ratio = self.crop_ratio

        crop_size = int(crop_ratio * input_size)
        if crop_size < self.out_size:
            crop_size = tv.transforms.transforms._setup_size(self.out_size,
                                                             error_msg="Please provide only two dimensions (h, w) for size.")
            i, j, h, w = transforms.RandomCrop.get_params(sample, crop_size)
            return TF.crop(sample, i, j, h, w)
        else:
            crop_size = tv.transforms.transforms._setup_size(crop_size,
                                                             error_msg="Please provide only two dimensions (h, w) for size.")
            i, j, h, w = transforms.RandomCrop.get_params(sample, crop_size)
            cropped = TF.crop(sample, i, j, h, w)
        out = TF.resize(cropped, self.out_size, self.interpolation)
                        
        return out
    

In [6]:
# Define the transforms to apply
transform = transforms.Compose([
    AdaptiveRandomCropTransform(crop_ratio=crop_ratio,
                                out_size=img_size_to_gpu,
                                interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.CenterCrop(size=384),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
                    
])

#invTrans = transforms.Compose([ 
#    transforms.Normalize(mean = [ 0., 0., 0. ], std = [ 1/0.229, 1/0.224, 1/0.225 ]), 
#    transforms.Normalize(mean = [ -0.485, -0.456, -0.406],  std = [ 1., 1., 1. ])])

In [9]:
from torch.utils.data import Dataset, DataLoader

class EPICImagesDataset(Dataset):
    def __init__(self, data_images, transform):
        self.data_images = data_images
        self.transform = transform

    def __len__(self):
        return len(self.data_images)

    def __getitem__(self, index):
        img_name = self.data_images.iloc[index]["distfilename"]

        try:
            img_np = imread(f"{base_path}/BiHealth/Data/EPICImages/{img_name}")
        except:
            img_np = imread(f"{base_path}/BiHealth/Data/EPICImages_PoorQuality/{img_name}")

        img_pil = PIL.Image.fromarray(img_np)

        try:
            img_tensor = self.transform(img_pil)
        except:
            print(img_name)

        return img_name, img_tensor

def collate_fn(batch):
    img_names, img_tensors = zip(*batch)
    img_names = list(img_names)
    img_tensors = torch.stack(img_tensors)
    return img_names, img_tensors

def predict_batch(model, img_batch):
    loghs = model(img_batch)["head_outputs"]["logits"].detach().cpu().numpy()
    return loghs

corrupted_files = ["0AIULA8E31FVNEXW_epiceye07142.png", 
                   "0AIULA8E315UZ3KA_epiceye03519.png", 
                   "0AIULA8E3354WXMB_epiceye03739.png",
                   "0AIULA8E32JU9I3E_epiceye00148.png",
                   "0AIULA8E3354JOZ0_epiceye06941.png",
                  "0AIULA8E315XYVCL_epiceye05788.png",
                   "0AIULA8E315WMUA2_epiceye05000.png",
                  "0AIULA8E32JRQLLF_epiceye01039.png",
                  "0AIULA8E329S6BCD_epiceye02546.png",
                  "0AIULA8E31RCZB57_epiceye00155.png",
                  "0AIULA8E31REEET3_epiceye05711.png",
                   "0AIULA8E32SJJSL9_epiceye03063.png",
                  "0AIULA8E31FQZ4OO_epiceye06657.png",
                   "0AIULA8E1HIF7IFB_epiceye05427.png",
                  "0AIULA8E32SDCBXM_epiceye00289.png",
                  "0AIULA8E1KEKKV8Y_epiceye05179.png"]

data_images = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/BiHealth_20230313_Long.dta").query("distfilename!=@corrupted_files")
dataset = EPICImagesDataset(data_images, transform)
#dataset = EPICImagesDataset(data_images, transform, base_path, num_workers=4, cache_size=100)
dataloader = DataLoader(dataset, batch_size=10, shuffle=False, num_workers=32, collate_fn=collate_fn, drop_last=False)
model = setup_training()

In [ ]:
partitions = [4, 5, 7, 9, 10, 20]


metadata = []
for iteration in tqdm(range(100)):
    for partition in tqdm(partitions):
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            ckpt = torch.load(f"{base_path}/BiHealth/ckpts/ckpt_partition_{partition}")
            model.encoder.load_state_dict({k[8:]:v for k, v in ckpt["state_dict"].items() if "encoder" in k}, strict=True)
            model.head.load_state_dict({k[5:]:v for k, v in ckpt["state_dict"].items() if "head" in k}, strict=True)
            model.eval();
            model.to("cuda")
            # instantiate cktp here
            for img_names, img_batch in dataloader:
                img_batch = img_batch.to("cuda")
                loghs = predict_batch(model, img_batch)
                for img_name, logh in zip(img_names, loghs):
                    metadata.append({"partition": partition, "img_name": img_name, "iteration": iteration, "loghs": logh})
                del img_batch

metadata_df = pd.DataFrame(metadata)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
metadata_df

,partition,img_name,iteration,loghs
0,0,0AIULA8E315X35Q2_epiceye00372.png,0,"[1.429, 3.213, 2.703, 3.55, 2.588, 0.71, 1.379..."
1,0,0AIULA8E1D5J220T_epiceye01548.png,0,"[1.355, 3.262, 2.824, 3.74, 2.777, 1.128, 2.06..."
2,0,0AIULA8E13CCINZ0_epiceye02465.png,0,"[-1.129, 1.634, 1.425, 1.722, 0.919, -0.8857, ..."
3,0,0AIULA8E2K7MOEX9_epiceye03852.png,0,"[1.787, 3.844, 3.35, 3.98, 2.81, 1.393, 1.391,..."
4,0,0AIULA8E21BAWQ50_epiceye07263.png,0,"[1.087, 3.055, 2.68, 3.285, 2.559, 1.223, 1.10..."
...,...,...,...,...
35257195,21,0AIULA8E24KW7A44_epiceye02489.png,99,"[0.91, 2.713, 2.277, 3.336, 2.38, 1.051, 1.541..."
35257196,21,0AIULA8E32S7E8OO_epiceye03011.png,99,"[0.9854, 2.914, 2.475, 3.766, 2.768, 1.403, 1...."
35257197,21,0AIULA8E2C2N6MYL_epiceye02574.png,99,"[1.303, 2.959, 2.47, 3.238, 2.443, 0.861, 1.56..."
35257198,21,0AIULA8E18C97Z2K_epiceye00650.png,99,"[1.362, 3.04, 2.678, 3.564, 2.36, 0.756, 1.9, ..."


In [11]:
metadata_df

,partition,img_name,iteration,loghs
0,4,0AIULA8E315X35Q2_epiceye00372.png,0,"[1.05, 2.055, 1.688, 2.31, 1.813, 0.5264, 1.09..."
1,4,0AIULA8E1D5J220T_epiceye01548.png,0,"[1.45, 2.574, 2.176, 2.744, 2.227, 0.7456, 1.3..."
2,4,0AIULA8E13CCINZ0_epiceye02465.png,0,"[0.3413, 1.57, 1.373, 1.723, 1.238, 0.005928, ..."
3,4,0AIULA8E2K7MOEX9_epiceye03852.png,0,"[1.27, 2.348, 2.014, 2.613, 1.878, 0.697, 0.87..."
4,4,0AIULA8E21BAWQ50_epiceye07263.png,0,"[0.5796, 1.634, 1.361, 1.937, 1.43, 0.4678, 0...."
...,...,...,...,...
9615595,20,0AIULA8E24KW7A44_epiceye02489.png,99,"[0.2869, 1.7295, 1.434, 2.186, 1.577, 0.753, 0..."
9615596,20,0AIULA8E32S7E8OO_epiceye03011.png,99,"[0.2458, 1.853, 1.625, 2.416, 1.78, 1.009, 1.2..."
9615597,20,0AIULA8E2C2N6MYL_epiceye02574.png,99,"[0.6025, 1.877, 1.547, 1.989, 1.453, 0.4739, 0..."
9615598,20,0AIULA8E18C97Z2K_epiceye00650.png,99,"[1.55, 2.775, 2.416, 3.174, 2.113, 0.8696, 2.1..."


In [12]:
metadata_df.reset_index(drop=True).to_feather(f"{base_path}/data/predictionstta_230629.feather")